In [3]:
import os
import sys
import difflib
import shutil
from glob import glob
import pandas as pd
import numpy as np
import operator

import music21

sys.path.append('..', '..')
from src.utils import get_chord_progressions, calculate_melody_results, flatten_chord_progression
from src.melody import Melody

source = 'Jazz-Midi'

input_folder = f'../data/Complete Examples Melodies Auto/v1.2/{source}'
filepaths = [y for x in os.walk(input_folder) for y in glob(os.path.join(x[0], '*.mid'))]
cps = get_chord_progressions('..')

In [8]:
file_idx = 8

In [9]:
filepaths[file_idx:100]

['../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/Eighty-One.mid',
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/Happy Birthday.mid',
 "../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/I Can't Give You Anything But Love.mid",
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/I Concentrate On You.mid',
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/I Should Care.mid',
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/In A Sentimental Mood.mid',
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/Moon River.mid',
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/My Funny Valentine.mid',
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/Night And Day.mid',
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/Our Love Is Here To Stay.mid',
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/People.mid',
 '../data/Complete Examples Melodies Auto/v1.2/Jazz-Midi/Polka Dots And Moonbeams.mid',
 '../data/Complete Exampl

In [11]:
filename = filepaths[file_idx]

all_hc = {}
for t in range(-6,6):
    melody = Melody(filename, '1.2')
    melody.setup()
    melody.set_song_structure(cps[melody.song_name])
    melody.align_key()
    
#     melody.tempo /= 2
    
    melody.transpose_semitones = t
    
    melody.parse_notes()

    # print(melody.note_info)

    if len(melody.errors) > 0:
        raise Exception(melody.errors[0])

    for starting_measure in range(0, melody.n_chord_prog_measures):
        melody.starting_measure = starting_measure
        melody.split_note_info = []

        melody.split_melody(quantized=True, save=False)

        results = calculate_melody_results(melody)
        
#         for s in melody.split_note_info:
#             s.loc[:, 'end_ticks'] = s['ticks'] + s['duration']
#             melody.remove_strong_polyphony(s)
#             melody.remove_weak_polyphony(s)

        hc = np.array([])
        for k, res in results.items():
            hc = np.append(hc, res['harmonic_consistency_mean']) 

        all_hc[str(t) + '_' + str(starting_measure)] = (hc.mean(), hc.max(), hc.min(), len(hc))
    
all_hc = sorted(all_hc.items(), key=operator.itemgetter(1), reverse=True)

for i in all_hc[:40]:
    print(i)
print('----------------')

('-5_10', (0.7847999999999999, 0.7847999999999999, 0.7847999999999999, 1))
('-5_14', (0.7770816326530612, 0.8481632653061225, 0.706, 2))
('-5_2', (0.7671052631578947, 0.7671052631578947, 0.7671052631578947, 1))
('-5_6', (0.7531914893617021, 0.7531914893617021, 0.7531914893617021, 1))
('-5_22', (0.738989898989899, 0.738989898989899, 0.738989898989899, 1))
('-5_18', (0.7348148148148148, 0.7348148148148148, 0.7348148148148148, 1))
('2_14', (0.7183124018838304, 0.8281632653061224, 0.6084615384615384, 2))
('2_21', (0.7167708333333334, 0.7167708333333334, 0.7167708333333334, 1))
('-3_20', (0.7126666666666668, 0.7126666666666668, 0.7126666666666668, 1))
('2_6', (0.7106382978723403, 0.7106382978723403, 0.7106382978723403, 1))
('-3_0', (0.7074074074074075, 0.7074074074074075, 0.7074074074074075, 1))
('2_2', (0.7028947368421053, 0.7028947368421053, 0.7028947368421053, 1))
('-3_12', (0.7018657635467981, 0.7155172413793104, 0.6882142857142858, 2))
('2_17', (0.7000270270270271, 0.7310000000000001, 

In [19]:
# tra_semi = all_hc[correct_index][0].split('_')[0]
# starting = all_hc[correct_index][0].split('_')[1]

correct_index = 0
tra_semi = -2
starting = 2

melody = Melody(filename, '1.2')
melody.setup()
melody.set_song_structure(cps[melody.song_name])
melody.align_key()

# melody.tempo /= 2

melody.transpose_semitones = int(tra_semi)
melody.starting_measure = int(starting)

melody.parse_notes()

melody.split_melody(quantized=True, save=False)

repetitions = len(melody.split_note_info)
# repetitions = 

print(melody.song_name)
print(f'Transpose semitones: {melody.transpose_semitones}')
print(f'Starting measure: {melody.starting_measure}')

print(f'N. repetitions: {repetitions}')

for r in range(1, repetitions + 1):
    melody.save_split_melody(repetition=r, quantized=True, chords=True)

Eighty-One
Transpose semitones: -2
Starting measure: 2
N. repetitions: 1


In [14]:
all_results = {}
results = calculate_melody_results(melody)

for k, v in results.items():
    all_results_key = melody.filename + ' -' + str(k) + '-'
    all_results[all_results_key] = v

In [15]:
gtg = input('Good to go? ')

if gtg == 'g':
    
    print('Finalising')
    
    for k, results in all_results.items():
        folder_midi = f'../data/split_melody/v1.2/{source}/'
        folder_csv = f'../data/split_melody_data/v1.2/{source}/'

        src_midi = folder_midi + results['out_filename']
        dst_midi = src_midi.replace('split_melody/v1.2', 'finalised/midi')

        src_csv = folder_csv + results['out_filename'].replace('.mid', '.csv')
        dst_csv = src_csv.replace('split_melody_data/v1.2', 'finalised/csv')

        if not os.path.exists(os.path.dirname(dst_midi)):
            os.makedirs(os.path.dirname(dst_midi))

        if not os.path.exists(os.path.dirname(dst_csv)):
            os.makedirs(os.path.dirname(dst_csv))

        shutil.copyfile(src_midi, dst_midi)
        shutil.copyfile(src_csv, dst_csv)

Good to go? g
Finalising


FileNotFoundError: [Errno 2] No such file or directory: '../data/split_melody/v1.2/Jazz-Midi/Eighty-One -1-.mid'